In [ ]:
"""Alunas: Clara Azevedo-GEC-1897; Maria Eduarda de Oliveira-GEC-1896; Vitória Dutra-GES-414"""

import threading
import random

class Player(threading.Thread):
    def __init__(self, player_id, name):
        """Inicializa um jogador com um ID e um nome."""
        super().__init__()
        self.player_id = player_id
        self.name = name
        self.choice = None
        self.opponent = None

    def make_choice(self):
        """Faz uma escolha aleatória entre 'pedra', 'papel' e 'tesoura'."""
        self.choice = random.choice(['pedra', 'papel', 'tesoura'])
        print(f'{self.name} escolheu {self.choice}')

    def run(self):
        """Método que será executado quando a thread for iniciada."""
        if self.opponent:
            self.make_choice()
            self.opponent.make_choice()

    def play_even_odd(self):
        """Joga par ou ímpar com outro jogador."""
        choice = random.choice(['par', 'ímpar'])
        number = random.randint(1, 10)
        print(f'{self.name} escolheu {choice} e jogou o número {number}')
        return choice, number

In [ ]:
import threading
import random
import time

class Player(threading.Thread):
    def __init__(self, name, age, opponent, winners_unsafe, wait_times, round_lock, decision_lock):
        super().__init__()
        self.name = name
        self.age = age
        self.opponent = opponent
        self.choice = None
        self.winners_unsafe = winners_unsafe  # Lista de ganhadores sem controle
        self.round_lock = round_lock
        self.wait_times = wait_times
        self.start_time = None
        self.winner_name = None
        self.decision_lock = decision_lock
        self.winner_decided = False  # Flag local por thread

    def make_choice(self):
        time.sleep(random.uniform(0.1, 0.4))
        self.choice = random.choice(['pedra', 'papel', 'tesoura'])
        print(f'{self.name} escolheu {self.choice}')

    def play_even_odd(self):
        time.sleep(random.uniform(0.1, 0.4))
        choice = random.choice(['par', 'ímpar'])
        number = random.randint(1, 10)
        print(f'{self.name} escolheu {choice} e jogou {number}')
        return choice, number

    def decide_winner(self):
        if self.choice == self.opponent.choice:
            print(f'{self.name} empatou com {self.opponent.name}, vão disputar par ou ímpar.')
            choice1, number1 = self.play_even_odd()
            choice2, number2 = self.opponent.play_even_odd()
            total = number1 + number2
            result = 'par' if total % 2 == 0 else 'ímpar'

            # Introdução de aleatoriedade para decidir o vencedor no caso de empate
            return self.name if random.choice([True, False]) else self.opponent.name
        else:
            if (self.choice == 'pedra' and self.opponent.choice == 'tesoura') or \
               (self.choice == 'papel' and self.opponent.choice == 'pedra') or \
               (self.choice == 'tesoura' and self.opponent.choice == 'papel'):
                return self.name
            else:
                return self.opponent.name

    def run(self):
        self.start_time = time.time()
        with self.round_lock:
            pass  # sincroniza início da rodada
        wait_duration = time.time() - self.start_time
        self.wait_times[self.name] = wait_duration
        print(f'{self.name} esperou {wait_duration:.3f} segundos antes de jogar')

        self.make_choice()
        self.opponent.make_choice()

        # Apenas uma thread por par decide o vencedor
        winner = None
        with self.decision_lock:
            if not self.winner_decided and not self.opponent.winner_decided:
                winner = self.decide_winner()
                self.winner_decided = True
                self.opponent.winner_decided = True
                self.winner_name = winner
                self.opponent.winner_name = winner

        # SEM CONTROLE — múltiplas threads podem adicionar o mesmo nome
        print(f'{self.name} tentando registrar (sem controle)...')
        self.winners_unsafe.append(self.winner_name)
        print(f'{self.name} registrou {self.winner_name} (sem controle).')

def simulate_game(player_data, scheduling='FCFS'):
    if scheduling == 'PS':
        player_data.sort(key=lambda x: -x[1])
        print("\n--- Escalonamento PS (por idade) ---")
    else:
        random.shuffle(player_data)
        print("\n--- Escalonamento FCFS ---")

    print("Jogadores:", [p[0] for p in player_data])

    player_names = [p[0] for p in player_data]
    player_ages = {p[0]: p[1] for p in player_data}

    round_num = 1
    start_time = time.time()

    while len(player_names) > 1:
        winners_unsafe = []  # Lista de ganhadores sem controle
        wait_times = {}
        round_lock = threading.Lock()
        decision_lock = threading.Lock()
        round_lock.acquire()
        threads = []

        for i in range(0, len(player_names) - 1, 2):
            name1 = player_names[i]
            name2 = player_names[i + 1]

            player1 = Player(name1, player_ages[name1], None, winners_unsafe, wait_times, round_lock, decision_lock)
            player2 = Player(name2, player_ages[name2], player1, winners_unsafe, wait_times, round_lock, decision_lock)
            player1.opponent = player2

            threads.append(player1)
            threads.append(player2)

        for t in threads:
            t.start()

        time.sleep(0.1)
        round_lock.release()

        for t in threads:
            t.join()

        print(f'\nGanhadores da rodada {round_num} (sem controle): {winners_unsafe}\n')

        new_player_names = []
        for name in winners_unsafe:
            if name not in new_player_names:
                new_player_names.append(name)

        player_names = new_player_names
        round_num += 1

    elapsed_time = time.time() - start_time
    print(f"\nTempo {scheduling}: {elapsed_time:.2f} segundos")
    print(f"\nVencedor: {player_names[0]}\n")

def main():
    players_with_ages = [
        ('Alice', 20), ('Joao', 25), ('Carla', 22), ('Davi', 28),
        ('Eva', 26), ('Lucas', 21), ('Julia', 23), ('Marcos', 24)
    ]

    simulate_game(players_with_ages, scheduling='FCFS')
    simulate_game(players_with_ages, scheduling='PS')

if __name__ == '__main__':
    main()



--- Escalonamento FCFS ---
Jogadores: ['Alice', 'Carla', 'Julia', 'Joao', 'Eva', 'Davi', 'Marcos', 'Lucas']
Alice esperou 0.101 segundos antes de jogar
Carla esperou 0.101 segundos antes de jogar
Julia esperou 0.101 segundos antes de jogar
Joao esperou 0.101 segundos antes de jogar
Eva esperou 0.101 segundos antes de jogar
Davi esperou 0.101 segundos antes de jogar
Marcos esperou 0.101 segundos antes de jogar
Lucas esperou 0.101 segundos antes de jogar
Lucas escolheu pedra
Marcos escolheu papel
Alice escolheu papel
Joao escolheu papel
Julia escolheu tesoura
Carla escolheu papel
Davi escolheu papel
Eva escolheu papel
Marcos escolheu papel
Lucas tentando registrar (sem controle)...
Lucas registrou Marcos (sem controle).
Eva escolheu tesoura
Davi tentando registrar (sem controle)...
Davi registrou Eva (sem controle).
Alice escolheu papel
Carla empatou com Alice, vão disputar par ou ímpar.
Lucas escolheu tesoura
Julia escolheu tesoura
Carla escolheu papel
Davi escolheu papel
Joao escolheu